<a href="https://colab.research.google.com/github/PG-2597/UIDAI-PySpark/blob/main/Assignment_Priya_Gupta_week15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Data Analysis on UIDAI-ENR-DETAIL Database using PySpark**

###**Installing and Initializing Pyspark and SparkContext:**

In [3]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 199 kB 35.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=db7cdf0be1025abc72f4dac08b5fb1dddd4c82b84c94d1d8cae93d1beefbc29e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [4]:
#Initializing PySpark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
#Spark Configuration
conf = SparkConf().setAppName("sample_app")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sqlContext = SQLContext(sc)


/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [5]:
sc

<SparkContext master=local[*] appName=sample_app>

###Reading the File and Loading The Data into a Dataframe:-

In [6]:
df = spark.read.format("csv").option("header", "true").option("inferSchema","true").load("/content/UIDAI-ENR-DETAIL-20170308.csv")

In [10]:
df.show(10)

+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|     Registrar|    Enrolment Agency|        State| District|Sub District|Pin Code|Gender|Age|Aadhaar generated|Enrolment Rejected|Residents providing email|Residents providing mobile number|
+--------------+--------------------+-------------+---------+------------+--------+------+---+-----------------+------------------+-------------------------+---------------------------------+
|Allahabad Bank|A-Onerealtors Pvt...|Uttar Pradesh|Allahabad|        Meja|  212303|     F|  7|                1|                 0|                        0|                                1|
|Allahabad Bank|Asha Security Gua...|Uttar Pradesh|Sonbhadra| Robertsganj|  231213|     M|  8|                1|                 0|                        0|                                0|
|Allahabad Bank|   SGS INDIA PVT LTD|Utt

###1.Create a dataframe with Total Aadhaar's generated for each state.

### If we see Aadhar generated column, there are values=0 as well. So, **first we need to filter our data.**

In [12]:
get_filtered_data=df.filter(df["Aadhaar generated"]>=1)

In [8]:
import pyspark.sql.functions as f

In [13]:
res=get_filtered_data.groupby("State").agg(f.sum("Aadhaar generated").alias("Total_Aadhar_Generated"))
res.show()

+-----------------+----------------------+
|            State|Total_Aadhar_Generated|
+-----------------+----------------------+
|         Nagaland|                    11|
|        Karnataka|                  1909|
|           Odisha|                   986|
|           Kerala|                    21|
|       Tamil Nadu|                  8653|
|     Chhattisgarh|                   758|
|   Andhra Pradesh|                   149|
|   Madhya Pradesh|                  1448|
|           Punjab|                  2651|
|          Manipur|                     6|
|              Goa|                     5|
|          Mizoram|                    19|
| Himachal Pradesh|                    12|
|       Puducherry|                     7|
|          Haryana|                   635|
|Jammu and Kashmir|                   161|
|        Jharkhand|                  1323|
|Arunachal Pradesh|                    17|
|          Gujarat|                   928|
|            Delhi|                  1802|
+----------

To find the total Aadhar's generated by each state, I used sum aggregate function and grouped it as per the state's name.

###2.Create a dataframe with the Total Aadhaar's generated by each enrollment agency.

In [14]:
get_filtered_data.groupby("Enrolment Agency").agg(f.sum("Aadhaar generated").alias("Total_Aadhar_Generated")).show()

+--------------------+----------------------+
|    Enrolment Agency|Total_Aadhar_Generated|
+--------------------+----------------------+
|    APOnline Limited|                   305|
|  Transmoovers India|                     5|
|     Alankit Limited|                    22|
|Twinstar Industri...|                   381|
|OSWAL COMPUTERS &...|                   578|
|SHRIKRISHNA KHAND...|                     1|
|Atalji Janasnehi ...|                  1369|
|Sri Ramraja Sarka...|                     1|
|Asha Security Gua...|                   164|
|Abha Systems And ...|                    69|
|               BASIX|                  8079|
|Frontech Systems ...|                   109|
|Vedavaag Systems ...|                     1|
|A I Soc for Elect...|                  6439|
|AVVAS INFOTECH PV...|                   256|
|      SARADA SYSTEMS|                     8|
|PROTEX COMPUTER P...|                  2180|
| Agro Tech Engineers|                   527|
|SREI INFRASTRUCTU...|            

###3. Create dataframe with the top 10 districts with maximum Aadhaar's generated for both Male and females?

#### Now, we need to filter our data again as per Gender.

In [15]:
res1= get_filtered_data.groupBy("District","Gender").sum("Aadhaar generated")

In [16]:
get_male_data= res1.filter(res1["Gender"]=="M")
get_male_top10= get_male_data.orderBy(f.desc("Sum(Aadhaar generated)")).limit(10)
get_male_top10.show()

+-----------------+------+----------------------+
|         District|Gender|sum(Aadhaar generated)|
+-----------------+------+----------------------+
|            Nadia|     M|                  1819|
|South 24 Parganas|     M|                  1439|
|        Bhagalpur|     M|                  1261|
|      Murshidabad|     M|                  1168|
|North 24 Parganas|     M|                  1068|
|       Barddhaman|     M|                  1033|
|           Jaipur|     M|                   906|
|          Katihar|     M|                   837|
|   Uttar Dinajpur|     M|                   833|
|         Khagaria|     M|                   829|
+-----------------+------+----------------------+



**Now, we will do the same for sorting females data as we did for males.**

In [17]:
get_female_data= res1.filter(res1["Gender"]=="F")
get_female_top10= get_female_data.orderBy(f.desc("Sum(Aadhaar generated)")).limit(10)
get_female_top10.show()

+-----------------+------+----------------------+
|         District|Gender|sum(Aadhaar generated)|
+-----------------+------+----------------------+
|           Budaun|     F|                  2360|
|North 24 Parganas|     F|                  1881|
|   Ramanathapuram|     F|                  1349|
|            Saran|     F|                   954|
|South 24 Parganas|     F|                   934|
|      Tirunelveli|     F|                   913|
|       Coimbatore|     F|                   835|
|            Karur|     F|                   819|
|            Patna|     F|                   638|
|            Nadia|     F|                   620|
+-----------------+------+----------------------+



###4. Create a dataframe with Total Aadhaar generated for the top 10 least states

In [20]:
res2=get_filtered_data.groupby("State").sum("Aadhaar generated")
least10_states= res2.orderBy(f.asc("Sum(Aadhaar generated)")).limit(10).show()

+--------------------+----------------------+
|               State|sum(Aadhaar generated)|
+--------------------+----------------------+
|              Others|                     1|
|Andaman and Nicob...|                     3|
|                 Goa|                     5|
|             Manipur|                     6|
|             Tripura|                     6|
|          Puducherry|                     7|
|           Meghalaya|                     9|
|            Nagaland|                    11|
|    Himachal Pradesh|                    12|
|   Arunachal Pradesh|                    17|
+--------------------+----------------------+



###5. For which age, most Aadhar cards have declined?

**Now, Aadhaar declined means value=0. So, here we will filter according to this condition.**

In [21]:
filter_aadhaar_decline=df.filter(df["Aadhaar generated"]<1)

In [22]:
res3=filter_aadhaar_decline.groupBy("Age").count()
final_res=res3.orderBy(f.desc("count")).limit(10)
final_res.show()

+---+-----+
|Age|count|
+---+-----+
|  4|  185|
|  3|  151|
|  2|  141|
|  1|  123|
|  5|  107|
|  6|   92|
|  0|   82|
|  7|   77|
|  8|   50|
|  9|   49|
+---+-----+

